In [ ]:
import os
import cv2
import numpy as np
# from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator

# Folder input
frame_dir = "../data/raw/frames"


# Folder output
base_dir = "../data/preprocessed"
rgb_dir = os.path.join(base_dir, "original_rgb")
norm_dir = os.path.join(base_dir, "normalized")
edge_dir = os.path.join(base_dir, "edged")
aug_dir = os.path.join(base_dir, "augmented")  # opsional

os.makedirs(rgb_dir, exist_ok=True)
os.makedirs(norm_dir, exist_ok=True)
os.makedirs(edge_dir, exist_ok=True)
os.makedirs(aug_dir, exist_ok=True)

# Data augmentasi (keras)
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True
)

# Loop frame
for i, frame in enumerate(frames):
    if i % 5 != 0:
        continue  # hanya setiap kelipatan 5

    resized = cv2.resize(frame, (224, 224))

    # 1. Simpan RGB asli
    cv2.imwrite(os.path.join(rgb_dir, f"frame_{i}.jpg"), resized)

    # 2. Normalisasi (0–1), simpan dalam bentuk uint8 agar bisa dilihat
    normalized = resized.astype("float32") / 255.0
    normalized_uint8 = (normalized * 255).astype("uint8")
    cv2.imwrite(os.path.join(norm_dir, f"frame_{i}.jpg"), normalized_uint8)

    # 3. Edge Detection (Canny)
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    cv2.imwrite(os.path.join(edge_dir, f"frame_{i}.jpg"), edges)

    # 4. Augmentasi (opsional, hanya 2 gambar)
    x = resized.reshape((1, *resized.shape))
    gen = datagen.flow(x, batch_size=1)
    for j in range(2):  # ambil 2 augmentasi
        aug = next(gen)[0].astype("uint8")
        cv2.imwrite(os.path.join(aug_dir, f"frame_{i}_aug{j}.jpg"), aug)

    print(f"Frame {i} processed.")
